In [54]:
import pickle
import os
from mmdetection3d import data, demo, configs, checkpoints
import open3d as o3d
from open3d.web_visualizer import draw
import numpy as np
import torch

## Load train infos

In [40]:
train_infos_path = os.path.join(
    data.__path__[0], 'kitti', 'kitti_infos_train.pkl')
train_infos = pickle.load(open(train_infos_path, 'rb'))

In [41]:
train_infos[2]['annos']

{'name': array(['Car', 'Car', 'Car', 'Cyclist', 'DontCare', 'DontCare'],
       dtype='<U8'),
 'truncated': array([ 0.,  0.,  0.,  0., -1., -1.]),
 'occluded': array([ 0,  0,  0,  0, -1, -1]),
 'alpha': array([ -1.56,   1.71,   1.64,   1.89, -10.  , -10.  ]),
 'bbox': array([[564.62, 174.59, 616.43, 224.74],
        [481.59, 180.09, 512.55, 202.42],
        [542.05, 175.55, 565.27, 193.79],
        [330.6 , 176.09, 355.61, 213.6 ],
        [753.33, 164.32, 798.  , 186.74],
        [738.5 , 171.32, 753.27, 184.42]]),
 'dimensions': array([[ 3.2 ,  1.61,  1.66],
        [ 3.7 ,  1.4 ,  1.51],
        [ 4.05,  1.46,  1.66],
        [ 1.95,  1.72,  0.5 ],
        [-1.  , -1.  , -1.  ],
        [-1.  , -1.  , -1.  ]]),
 'location': array([[-6.900e-01,  1.690e+00,  2.501e+01],
        [-7.430e+00,  1.880e+00,  4.755e+01],
        [-4.710e+00,  1.710e+00,  6.052e+01],
        [-1.263e+01,  1.880e+00,  3.409e+01],
        [-1.000e+03, -1.000e+03, -1.000e+03],
        [-1.000e+03, -1.000e+03, -

## Inference

In [101]:
from mmdet3d.apis import inference_detector, init_model
from mmdet3d.core.points import get_points_type
from mmdet3d.datasets.pipelines.loading import LoadPointsFromFile

In [109]:
import sensus

In [110]:
sensus.utils.data_converter.pc2points_class

AttributeError: module 'sensus' has no attribute 'utils'

In [71]:
# build the model from a config file and a checkpoint file
model_cfg = os.path.join(configs.__path__[0],
    'pointpillars/hv_pointpillars_secfpn_6x8_160e_kitti-3d-3class.py')
checkpoint_path = os.path.join(checkpoints.__path__[0],
    'hv_pointpillars_secfpn_6x8_160e_kitti-3d-3class_20220301_150306-37dc2420.pth')
pcd_path = os.path.join(demo.__path__[0],
    'data/kitti/kitti_000008.bin')
device = 'cuda:0'
model = init_model(model_cfg, checkpoint_path, device=device)

load checkpoint from local path: /home/messi/alvaro/sensus-loci/mmdetection3d/checkpoints/hv_pointpillars_secfpn_6x8_160e_kitti-3d-3class_20220301_150306-37dc2420.pth


/home/messi/alvaro/sensus-loci/mmdetection3d/mmdet3d/models/dense_heads/anchor3d_head.py:84: UserWarning: dir_offset and dir_limit_offset will be depressed and be incorporated into box coder in the future
  warnings.warn(


In [20]:
# test a single image
result, data = inference_detector(model, pcd_path)

In [103]:
with open(pcd_path, 'rb') as f:
    points = np.fromfile(f, dtype=np.float32, count=-1)

In [80]:
tmp = {**model.cfg.data.test.pipeline[0]}
tmp.pop('type')
tmp

{'coord_type': 'LIDAR',
 'load_dim': 4,
 'use_dim': 4,
 'file_client_args': {'backend': 'disk'}}

In [104]:
point_loader = LoadPointsFromFile(**tmp)
# points = point_loader._load_points(pcd_path)

In [105]:
points = points.reshape(-1, point_loader.load_dim)
points = points[:, point_loader.use_dim]
attribute_dims = None

if point_loader.shift_height:
    floor_height = np.percentile(points[:, 2], 0.99)
    height = points[:, 2] - floor_height
    points = np.concatenate(
        [points[:, :3],
            np.expand_dims(height, 1), points[:, 3:]], 1)
    attribute_dims = dict(height=3)

if point_loader.use_color:
    assert len(point_loader.use_dim) >= 6
    if attribute_dims is None:
        attribute_dims = dict()
    attribute_dims.update(
        dict(color=[
            points.shape[1] - 3,
            points.shape[1] - 2,
            points.shape[1] - 1,
        ]))

points_class = get_points_type(point_loader.coord_type)
points = points_class(
    points, points_dim=points.shape[-1], attribute_dims=attribute_dims)

In [106]:
points

LiDARPoints(
    tensor([[ 2.1554e+01,  2.8000e-02,  9.3800e-01,  3.4000e-01],
        [ 2.1240e+01,  9.4000e-02,  9.2700e-01,  2.4000e-01],
        [ 2.1056e+01,  1.5900e-01,  9.2100e-01,  5.3000e-01],
        ...,
        [ 6.3150e+00, -3.1000e-02, -1.6490e+00,  2.9000e-01],
        [ 6.3090e+00, -2.1000e-02, -1.6470e+00,  2.9000e-01],
        [ 6.3110e+00, -1.0000e-03, -1.6480e+00,  3.2000e-01]]))

In [107]:
# %%timeit
result, data = inference_detector(model, points)

In [108]:
result

[{'boxes_3d': LiDARInstance3DBoxes(
      tensor([[ 14.7599,  -1.1046,  -1.5370,   3.7431,   1.5425,   1.4897,  -0.3122],
          [  8.1152,   1.2280,  -1.5825,   3.6387,   1.5505,   1.5314,   2.8387],
          [  6.4760,  -3.8702,  -1.7344,   3.1718,   1.5082,   1.4379,  -0.2825],
          [ 33.6069,  -7.0866,  -1.3756,   4.3355,   1.7495,   1.7403,   2.8679],
          [  3.7632,   2.7247,  -1.6718,   3.6459,   1.5722,   1.5840,  -0.2663],
          [ 20.1171,  -8.4429,  -1.7343,   2.2501,   1.4856,   1.5236,  -0.3569],
          [ 24.9543, -10.2091,  -1.7078,   3.8045,   1.6183,   1.4614,  -0.4060],
          [ 28.7499,  -1.5279,  -1.2304,   4.1485,   1.6273,   1.5672,   1.2427],
          [ 55.5492, -20.3064,  -1.3923,   4.3676,   1.7144,   1.7606,   2.7885],
          [ 40.9051,  -9.8465,  -1.3449,   3.8703,   1.5952,   1.5452,  -0.3638]])),
  'scores_3d': tensor([0.9519, 0.9455, 0.9103, 0.8713, 0.8360, 0.7961, 0.7738, 0.6633, 0.6452,
          0.5037]),
  'labels_3d': tensor(

## Visualization

In [34]:
bin_path = os.path.join(demo.__path__[0],
    'data/kitti/kitti_000008.bin')
pc_path = os.path.join(
    '/home/messi/alvaro/sensus-loci/mmdetection3d/demo/kitti_pointpillars/kitti_000008',
    'kitti_000008_points.obj')
bboxes_path = os.path.join(
    '/home/messi/alvaro/sensus-loci/mmdetection3d/demo/kitti_pointpillars/kitti_000008',
    'kitti_000008_pred.obj')

In [51]:
# Read pc from bin file
with open(pcd_path, 'rb') as f:
    points = np.fromfile(f, dtype=np.float32, count=-1).reshape([-1, 4])

pcd_bin = o3d.geometry.PointCloud()
pcd_bin.points = o3d.utility.Vector3dVector(points[:, :3])
print(pcd_bin)

PointCloud with 17238 points.


In [37]:
# Read pc from obj file (reading with o3d.io.read_triangle_mesh or 
# read_point_cloud does not work)
pc = []
with open(pc_path, 'rb') as f:
    for each in f.readlines():
        p1, p2, p3 = each.decode('utf-8').split(' ')[1:]
        pc.append([float(p1), float(p2), float(p3.replace('\n', ''))])

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(np.array(pc))
print(pcd)

bboxes = o3d.io.read_triangle_mesh(bboxes_path)
bboxes.compute_vertex_normals()     # For solid rendering with lighting
bboxes_lines = o3d.geometry.LineSet().create_from_triangle_mesh(bboxes)
bboxes_lines.paint_uniform_color([1, 0, 0])
print(bboxes_lines)

PointCloud with 16897 points.
LineSet with 360 lines.


In [29]:
draw([pcd, bboxes_lines], width=900, height=600, point_size=2)

WebVisualizer(window_uid='window_11')

In [52]:
draw([pcd_bin, bboxes_lines], width=900, height=600, point_size=2)

WebVisualizer(window_uid='window_0')

# Trials (not working yet)

## Visualization with meshio

In [ ]:
! pip install meshio -y

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 22.11.1
  latest version: 23.1.0

Please update conda by running

    $ conda update -n base -c defaults conda

Or to minimize the number of packages updated during conda update use

     conda install conda=23.1.0



## Package Plan ##

  environment location: /home/messi/anaconda3

  added / updated specs:
    - meshio


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    cftime-1.5.1.1             |   py38hce1f21e_0         206 KB
    conda-22.11.1              |   py38h578d9bd_1         905 KB  conda-forge
    hdf4-4.2.13                |       h3ca952b_2         714 KB
    libnetcdf-4.8.1            |       h8322cc2_2         1.2 MB
    libzip-1.8.0               |       h5cef20c_0         102 KB
    markdown-it-py-2.1.0       |     pyhd8ed1ab_0          57 KB  conda-forge
    mdurl-0.1.0   

In [ ]:
import meshio
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import numpy as np

In [ ]:
msh = meshio.read(pc_path)
verts = msh.points
middle = np.max(verts, axis=0) + np.min(verts, axis=0)/2
verts = verts - middle
x, y, z = verts.T

In [ ]:
print(x.min(),x.max())
print(y.min(),y.max())
print(z.min(),z.max())

-31.559000000000005 5.138999999999999
-65.932499 -1.444500000000005
-1.9 0.902


In [ ]:
trace = go.Scatter3d(x=x, y=y, z=z,
        mode='markers', marker=dict(size=1, color=z, colorscale='Viridis', opacity=0.8))
layout = go.Layout(title = '3D Scatter plot', width = 800, height = 800)
fig = go.Figure(data = [trace], layout = layout)
fig.show()